![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Clinical Deidentification

In [3]:
import os

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl


2.4.2

In [4]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.230.13:7077 
 AppName 
 Databricks Shell

# Deidentification Model

In [6]:
import pandas as pd
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel
import pyspark.sql.functions as F
import string
import numpy as np
import sparknlp
from sparknlp.util import *
from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F
from sparknlp_jsl.annotator import *

In [7]:
from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = NerDLModel.pretrained("deidentify_dl", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
deidentify_dl download started this may take some time.
Approximate size to download 14.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [8]:
deidentification_rules = DeIdentificationModel.pretrained("deidentify_rb", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("deidentified")


deidentify_rb download started this may take some time.
Approximate size to download 3.8 KB
[ | ][ / ][ — ][OK!]

In [9]:
def get_deidentify_model():
    
    custom_ner_converter = NerConverter()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk")
      #.setWhiteList(entity_types)
    
    deidentify_pipeline = Pipeline(
            stages = [
            documentAssembler,
            sentenceDetector,
            tokenizer,
            word_embeddings,
            clinical_ner,
            custom_ner_converter,
            deidentification_rules
            ])

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    model_deidentify = deidentify_pipeline.fit(empty_data)
    
    return model_deidentify



In [10]:
deid_model = get_deidentify_model()

In [11]:

def get_deidentified_text(model, text):
        
    light_model = LightPipeline(model)
    
    annotated_text = light_model.annotate(text)
    
    return annotated_text['deidentified']
   

In [12]:
text ='''
A . Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01/13/93 PCP : Oliveira , 25 month years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''

In [13]:
light_model = LightPipeline(deid_model)
    
annotated_text = light_model.annotate(text)
annotated_text

Out[16]: {'document': ['\nA . Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01/13/93 PCP : Oliveira , 25 month years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street\n'],
 'ner_chunk': ['2093-01-13',
 'David Hale',
 'Hendrickson',
 'Ora',
 '7194334',
 '01/13/93',
 'Oliveira',
 '2079-11-09',
 'Cocke County Baptist Hospital',
 '0295 Keats Street'],
 'deidentified': ['A .',
 'Record date : 2093-01-13 , <DOCTOR> , M.D .',
 ', Name : <PATIENT> , <PATIENT> MR .',
 '# <MEDICALRECORD> Date : <DATE> PCP : <DOCTOR> , 25 month years-old , Record date : 2079-11-09 .',
 '<HOSPITAL> .',
 '<STREET>'],
 'token': ['A',
 '.',
 'Record',
 'date',
 ':',
 '2093-01-13',
 ',',
 'David',
 'Hale',
 ',',
 'M.D',
 '.',
 ',',
 'Name',
 ':',
 'Hendrickson',
 ',',
 'Ora',
 'MR',
 '.',
 '#',
 '7194334',
 'Date',
 ':',
 '01/13/93',
 'PCP',
 ':',
 'Oliveira',
 ',',
 '25',
 'month',
 'years-old',
 ',',
 'Record',
 'date',
 ':',
 '2079-11-09',
 '.',
 'Cocke',
 'County',
 'Baptist',
 'Hospital',
 '.',
 '0295',
 'Keats',
 'Street'],
 'ner': ['O',
 'O',
 'O',
 'O',
 'O',
 'I-DATE',
 'O',
 'I-DOCTOR',
 'I-DOCTOR',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-PATIENT',
 'O',
 'I-PATIENT',
 'O',
 'O',
 'O',
 'I-MEDICALRECORD',
 'O',
 'O',
 'I-DATE',
 'O',
 'O',
 'I-DOCTOR',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-DATE',
 'O',
 'I-HOSPITAL',
 'I-HOSPITAL',
 'I-HOSPITAL',
 'I-HOSPITAL',
 'O',
 'I-STREET',
 'I-STREET',
 'I-STREET'],
 'embeddings': ['A',
 '.',
 'Record',
 'date',
 ':',
 '2093-01-13',
 ',',
 'David',
 'Hale',
 ',',
 'M.D',
 '.',
 ',',
 'Name',
 ':',
 'Hendrickson',
 ',',
 'Ora',
 'MR',
 '.',
 '#',
 '7194334',
 'Date',
 ':',
 '01/13/93',
 'PCP',
 ':',
 'Oliveira',
 ',',
 '25',
 'month',
 'years-old',
 ',',
 'Record',
 'date',
 ':',
 '2079-11-09',
 '.',
 'Cocke',
 'County',
 'Baptist',
 'Hospital',
 '.',
 '0295',
 'Keats',
 'Street'],
 'sentence': ['A .',
 'Record date : 2093-01-13 , David Hale , M.D .',
 ', Name : Hendrickson , Ora MR .',
 '# 7194334 Date : 01/13/93 PCP : Oliveira , 25 month years-old , Record date : 2079-11-09 .',
 'Cocke County Baptist Hospital .',
 '0295 Keats Street']}

In [14]:
get_deidentified_text(deid_model, text)

Out[11]: ['A .',
 'Record date : 2093-01-13 , <DOCTOR> , M.D .',
 ', Name : <PATIENT> , <PATIENT> MR .',
 '# <MEDICALRECORD> Date : <DATE> PCP : <DOCTOR> , 25 month years-old , Record date : 2079-11-09 .',
 '<HOSPITAL> .',
 '<STREET>']

In [15]:
source_text = '''Record date: 2093-01-13 David Hale, M.D.
Name: Hendrickson, Ora MR. #7194334 Date: 01/13/93 PCP: Oliveira.
Record date: 2079-11-09. Cocke County Baptist Hospital. 0295 Keats Street.
This 17-yr-old male, presented with chest heaviness that started during a pick-up basketball game. His past medical history was unremarkable. He denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age 38. The patient was a nonsmoker, did not drink alcohol, and denied recreational drug use. He swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal.'''

data = spark.createDataFrame([
  [source_text]
]).toDF("text")

data.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
text |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Record date: 2093-01-13 David Hale, M.D.
Name: Hendrickson, Ora MR. #7194334 Date: 01/13/93 PCP: Oliveira.
Record date: 2079-11-09. Cocke County Baptist Hospital. 0295 Keats Street.
This 17-yr-old male, presented with chest heaviness that started during a pick-up basketball game. His past medical history was unremarkable. He denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age 38. The patient was a nonsmoker, did not drink alcohol, and denied recreational drug use. He swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal.|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [16]:
result = deid_model.transform(data)

In [17]:

result.select(F.explode("deidentified.result")).show(truncate=False)

+--------------------------------------------------------------------------------------------------+
col |
+--------------------------------------------------------------------------------------------------+
Record date: 2093-01-13 David Hale, M.D. |
Name: Hendrickson, <PATIENT> MR. #7194334 Date: <DATE> PCP: <PATIENT>. |
Record date: 2079-11-09. |
<HOSPITAL>. |
<STREET>. |
This 17-yr-old male, presented with chest heaviness that started during a pick-up basketball game.|
His past medical history was unremarkable. |
He denied prior cardiac symptoms and suffered no chest trauma during the game. |
His father had suffered an acute myocardial infarction at age |
38. The patient was a nonsmoker, did not drink alcohol, and denied recreational drug use. |
He swallowed a tablet of aspirin before coming to the emergency room. |
His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. |
Physical examination revealed no stigmata of Marfan syndrome. |
The rest of his physical examination was normal. |
+--------------------------------------------------------------------------------------------------+